In [18]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
import linmix

'''This code takes our sample and calculates a regression line that statistically factors in null data points.'''

#1. This section imports the necessary columns from the csv file into arrays
csv = np.genfromtxt ('RadioDetectionTable_Limits.csv', delimiter=",")
spectype = csv[1:,2] 
LV = csv[1:,7]
ysig = csv[1:,8]
xsig = csv[1:,9]
delta = csv[1:,10]
delta = delta.astype(int)
delta = delta == 0


In [2]:
#THIS RUNS WITHOUT LV IN LOG FORMAT
#--. Runs the linmix algorithm (uses markov chain monte carlo)
#--. Plots two subplots. The right is the sample, and the left is the sample with error measurements and linear regression running through it.
# lm = linmix.LinMix(spectype, LV, xsig, ysig, K=2)
# lm.run_mcmc(silent=True)

# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(121)
# ax.scatter(spectype, LV)
# ax.set_xlabel('Spectral Type')
# ax.set_ylabel('L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
# # ax.set_yscale('log')
# ax.set_xlim(6,29)
# ax.set_ylim(400000000000,650000000000000)
# ax = fig.add_subplot(122)
# ax.scatter(spectype, LV, alpha=0.5)
# #ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
# for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
#     xs = np.arange(6,31)
#     ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
#     ax.plot(xs, ys, color='r', alpha=0.07)
# #     return alpha_av = np.mean(lm.chain[i]['alpha'])
# #     return beta_av = np.mean(lm.chain[i]['beta'])
# # ys = alpha_av + xs * beta_av
# # ax.plot(xs, ys, color='k')
# ax.set_xlabel('Spectral Type')
# ax.set_ylabel('L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
# # ax.set_yscale('log')
# ax.set_xlim(6,29)
# # ax.set_ylim(400000000000,1000000000000000)
# ax.set_ylim(400000000000,650000000000000)
# ax.plot([6,29,29,6,6], [400000000000,400000000000,1000000000000000,1000000000000000,400000000000], color='k')
# fig.tight_layout()
# fig.savefig('radio_linmixanalysis.png')

In [16]:
linmix.LinMix?

In [19]:
#2. Converts Radio flux to log_10 values of radio flux
LV=np.log10(LV)

In [2]:
# print LV

In [20]:
#3. Runs the linmix algorithm (uses markov chain monte carlo)

lm = linmix.LinMix(spectype, LV, xsig=None, ysig=None, K=2)
lm.run_mcmc(silent=False)


Iteration:  100
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 0.9948006   0.99410836  0.99749877  1.00126277  0.99789292  0.99359465]

Iteration:  200
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00055441  1.00014237  1.00400098  0.99952291  1.0021516   1.00089855]

Iteration:  300
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00220844  1.00272602  1.00155228  1.00040876  0.99949601  1.00251027]

Iteration:  400
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.0005668   0.9995174   1.00120123  0.99986231  1.0004676   0.99939523]

Iteration:  500
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00013906  0.99911176  1.0002209   0.99863408  1.00088461  0.99916489]

Iteration:  600
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, e

In [21]:
#4. This is the same action as 4 above, just the flux is in log_10
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.scatter(spectype, LV)
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
          'T0','','T2','','T4','','T6','','T8','']
ylabels = ['','11','','13','','15','','17','']
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
ax = fig.add_subplot(122)
ax.scatter(spectype, LV, alpha=0.5)
#ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
    ax.plot(xs, ys, color='r', alpha=0.07)
alpha_av = lm.chain['alpha'].mean()
beta_av = lm.chain['beta'].mean()
ys = alpha_av + xs * beta_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
fig.tight_layout()
fig.savefig('radio_linmixanalysislogged.png')
plt.close(fig)

In [46]:
#5. This runs the mcmc code for only confirmed detections
# Can be commented out, creates arrays that will only show upper limits
spectype_conf=spectype[delta]
LV_conf=LV[delta]

lmconf = linmix.LinMix(spectype_conf, LV_conf, xsig=None, ysig=None, K=2)
lmconf.run_mcmc(silent=False)


Iteration:  100
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 0.99461861  0.99256704  1.00466647  0.99718649  0.99386981  0.99238596]

Iteration:  200
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00110975  1.00006571  0.99651004  1.00070702  0.99947315  1.00045176]

Iteration:  300
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00353393  1.00202303  0.99744632  1.00586212  0.99946698  1.00267889]

Iteration:  400
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00005481  1.00106899  0.99805743  1.00029167  0.99861209  1.00073564]

Iteration:  500
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 0.99963739  1.0002396   0.99889775  0.99860971  0.99881675  1.00025753]

Iteration:  600
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, e

In [23]:
#6. This runs the mcmc code with delta indicating upper limits. 0=confirmed detections 1=null detections
lmcens  = linmix.LinMix(spectype, LV, xsig, ysig, delta=delta, K=2)
lmcens.run_mcmc(silent=False)


Iteration:  100
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.00286334  1.00225087  1.04602974  0.9904924   0.99631509  0.99924846]

Iteration:  200
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.02115741  1.01597869  1.07825822  1.00638744  0.997831    1.01555166]

Iteration:  300
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.01326119  1.01854227  1.07832962  1.00860148  0.99937147  1.01549771]

Iteration:  400
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.03616074  1.03191986  1.04045352  1.00511143  0.99878435  1.03046639]

Iteration:  500
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.01700445  1.01359503  1.0225704   1.00753296  0.99957314  1.01326955]

Iteration:  600
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, e

In [49]:
#7. This runs the new plots with upper limits factored in

#Defines not delta to graph correct arrays
notdelta = np.logical_not(delta)
labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
          'T0','','T2','','T4','','T6','','T8','']
ylabels = ['','11','','13','','15','','17','']

#Same as 5, plots the left side of subplot
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(221)
ax.errorbar(spectype[delta], LV[delta], ls='', c='k', fmt="o")
ax.errorbar(spectype[notdelta], LV[notdelta], ls='', c='grey', alpha=0.2, fmt="v")
for i in xrange(0, len(lmconf.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lmconf.chain[i]['alpha'] + xs * lmconf.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphaconf_av = lmconf.chain['alpha'].mean()
betaconf_av = lmconf.chain['beta'].mean()
ys = alphaconf_av + xs * betaconf_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
ax.annotate('slope:' + str(betaconf_av) + '+/-' + str(lmconf.chain['beta'].std()), (7,17))
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)

#Now the middle plot
ax = fig.add_subplot(222)
ax.scatter(spectype, LV, c='k', alpha=0.5)
# ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alpha_av = lm.chain['alpha'].mean()
beta_av = lm.chain['beta'].mean()
ys = alpha_av + xs * beta_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)

#Now the right side plot, this has upper limits
ax = fig.add_subplot(223)
ax.errorbar(spectype[delta], LV[delta], ls='', c='k', fmt="o")
ax.errorbar(spectype[notdelta], LV[notdelta], yerr=ysig[notdelta],uplims=np.ones(sum(notdelta), dtype=bool), ls='', c='r', alpha=0.4, fmt="v")
for i in xrange(0, len(lmcens.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6, 31)
    ys = lmcens.chain[i]['alpha'] + xs * lmcens.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphacens_av = lmcens.chain['alpha'].mean()
betacens_av = lmcens.chain['beta'].mean()
ys = alphacens_av + xs * betacens_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
ax.annotate('slope:' + str(betacens_av) + '+/-' + str(lmcens.chain['beta'].std()), (7,17))
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
fig.tight_layout()
fig.savefig('radio_linmixanalysis_upperlims.png')
plt.close(fig)
plt.show()

print alphacens_av, lmcens.chain['alpha'].std()
print betacens_av,  lmcens.chain['beta'].std()

print alphaconf_av, lmconf.chain['alpha'].std()
print betaconf_av,  lmconf.chain['beta'].std()

13.2862328576 0.420535199612
0.00469862203964 0.0281131502805
13.2656744774 0.426833008143
0.005405398382 0.0289345958389


In [3]:
# LV_Confirmed=LV[np.all([delta < 1], axis=0)]
# print LV_Confirmed

# LV_UL=LV[np.all([delta equal 1], axis =0)]
# print LV_UL

In [47]:
print ysig_conf

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
